<a href="https://colab.research.google.com/github/luizthompson/MVP_Sprints/blob/main/MVP_Sprint_Machine_Learning_%26_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Sprint: Machine Learning & Analytics
Luiz Guilherme Thompson Vaz


# Definição do Problema

Descrição do Problema:
O problema em questão é analisar os resultados de partidas da Copa Libertadores da América, considerando informações como a edição do torneio, a rodada, a data da partida, os clubes envolvidos e os placares.

# Premissas ou Hipóteses


Os resultados das partidas refletem o desempenho das equipes.
As características dos clubes (força, estratégias, histórico, etc.) influenciam os resultados.
O formato do torneio permaneceu constante ao longo das edições analisadas.

# Restrições ou Condições para Selecionar os Dados

Os dados devem ser confiáveis e abranger várias edições da Copa Libertadores.
Os dados devem conter informações relevantes para análise, como os clubes envolvidos, as datas e os placares das partidas.

# Descrição do Dataset
O dataset consiste em um conjunto de dados tabulares com as seguintes colunas:

Edition: Ano da edição da Copa Libertadores.
Round: Rodada da partida (ex: Final, Semifinal, Quartas de Final, etc.).
Date: Data da partida.
Home Club: Clube mandante.
Away Club: Clube visitante.
Home Score: Placar do clube mandante.
Away Score: Placar do clube visitante.


In [ ]:
# Importar as bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Definição do Problema

# Objetivo: entender e descrever claramente o problema que está sendo resolvido.

# Qual é a descrição do problema?
# Você tem premissas ou hipóteses sobre o problema? Quais?
# Que restrições ou condições foram impostas para selecionar os dados?
# Descreva o seu dataset (atributos, imagens, anotações, etc).

# URL do arquivo CSV no GitHub
url_dados = 'https://raw.githubusercontent.com/luizthompson/MVP_Sprints/8dd5432382946862109f439d0a49642b781a45e7/libertadores-results-ds.csv'

# Carregar o dataset através do CSV
df = pd.read_csv(url_dados)

# Visualizar as primeiras linhas do dataset
print("Primeiras linhas do dataset:")
print(df.head())



Primeiras linhas do dataset:
   Edition      Round        Date      Home Club      Away Club  Home Score  \
0     2023      Final   4/11/2023  Fluminense FC   Boca Juniors           2   
1     2023  Semifinal   6/10/2023      Palmeiras   Boca Juniors           1   
2     2023  Semifinal   5/10/2023  Internacional  Fluminense FC           1   
3     2023  Semifinal  29/09/2023   Boca Juniors      Palmeiras           0   
4     2023  Semifinal  28/09/2023  Fluminense FC  Internacional           2   

   AwayScore  
0          1  
1          1  
2          2  
3          0  
4          2  


# Preparação de Dados

Separação do Dataset
O dataset pode ser separado em conjuntos de treino e teste, onde os dados de uma determinada edição podem ser utilizados para treino e os dados de uma edição posterior podem ser usados para teste.


In [ ]:
# Separar os atributos (features) e os rótulos (target)
X = df.drop(['Home Score', 'AwayScore'], axis=1)  # atributos
y = df[['Home Score', 'AwayScore']]  # rótulos

# Dividir o dataset em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Validação Cruzada
Dado que temos um conjunto de dados históricos, faz sentido utilizar a validação cruzada para avaliar a performance dos modelos, especialmente considerando que diferentes edições da Copa Libertadores podem ter características distintas.

# Transformações de Dados

Normalização dos placares das partidas para que estejam na mesma escala.
Codificação das variáveis categóricas como Round, Home Club e Away Club.
Divisão dos dados em atributos (features) e rótulos (target).

In [ ]:
# Normalizar os placares das partidas
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)


# Modelagem e Treinamento

Algoritmos Indicados:
Algoritmos de classificação podem ser adequados para prever se um determinado time vencerá uma partida ou não, como Regressão Logística, Árvores de Decisão, Random Forest, entre outros.



In [ ]:
# Inicializar e treinar o modelo de Regressão Logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train_encoded, y_train_scaled.ravel())


NameError: name 'LogisticRegression' is not defined